In [1]:
import numpy as np
import random
import copy
from scipy.stats import bernoulli
#from tqdm.notebook import tqdm

In [2]:
class Environment(object):
    '''General RL environment'''

    def __init__(self):
        pass

    def reset(self):
        pass

    def advance(self, action):
        '''
        Moves one step in the environment.
        Args:
            action
        Returns:
            reward - double - reward
            newState - int - new state
            pContinue - 0/1 - flag for end of the episode
        '''
        return 0, 0, 0

In [3]:
def make_riverSwim(epLen=20, nState=5):
    '''
    Makes the benchmark RiverSwim MDP.
    Args:
        NULL - works for default implementation
    Returns:
        riverSwim - Tabular MDP environment '''
    nAction = 2
    R_true = {}
    P_true = {}
    states = {}
    for s in range(nState):
        states[(s)] = 0.0
        for a in range(nAction):
            R_true[s, a] = (0, 0)
            P_true[s, a] = np.zeros(nState)

    # Rewards
    R_true[0, 0] = (5/1000, 0)
    R_true[nState - 1, 1] = (1, 0)

    # Transitions
    for s in range(nState):
        P_true[s, 0][max(0, s-1)] = 1.

    for s in range(1, nState - 1):
        P_true[s, 1][min(nState - 1, s + 1)] = 0.3
        P_true[s, 1][s] = 0.6
        P_true[s, 1][max(0, s-1)] = 0.1

    P_true[0, 1][0] = 0.3
    P_true[0, 1][1] = 0.7
    P_true[nState - 1, 1][nState - 1] = 0.9
    P_true[nState - 1, 1][nState - 2] = 0.1

    riverSwim = TabularMDP(nState, nAction, epLen)
    riverSwim.R = R_true
    riverSwim.P = P_true
    riverSwim.states = states
    riverSwim.reset()

    return riverSwim

class TabularMDP(Environment):
    '''
    Tabular MDP
    R - dict by (s,a) - each R[s,a] = (meanReward, sdReward)
    P - dict by (s,a) - each P[s,a] = transition vector size S
    '''

    def __init__(self, nState, nAction, epLen):
        '''
        Initialize a tabular episodic MDP
        Args:
            nState  - int - number of states
            nAction - int - number of actions
            epLen   - int - episode length
        Returns:
            Environment object
        '''

        self.nState = nState
        self.nAction = nAction
        self.epLen = epLen

        self.timestep = 0
        self.state = 0

        # Now initialize R and P
        self.R = {}
        self.P = {}
        self.states = {}
        for state in range(nState):
            for action in range(nAction):
                self.R[state, action] = (1, 1)
                self.P[state, action] = np.ones(nState) / nState
                
    def reset(self):
        "Resets the Environment"
        self.timestep = 0
        self.state = 0
        
    def advance(self,action):
        '''
        Move one step in the environment
        Args:
        action - int - chosen action
        Returns:
        reward - double - reward
        newState - int - new state
        episodeEnd - 0/1 - flag for end of the episode
        '''
        if self.R[self.state, action][1] < 1e-9:
            # Hack for no noise
            reward = self.R[self.state, action][0]
        else:
            reward = np.random.normal(loc=self.R[self.state, action][0],
                                      scale=self.R[self.state, action][1])
        #print(self.state, action, self.P[self.state, action])
        newState = np.random.choice(self.nState, p=self.P[self.state, action])
        
        # Update the environment
        self.state = newState
        self.timestep += 1

        episodeEnd = 0
        if self.timestep == self.epLen:
            episodeEnd = 1
            #newState = None
            self.reset()

        return reward, newState, episodeEnd
    
    def argmax(self,b):
        #print(b)
        return np.random.choice(np.where(b == b.max())[0])

In [4]:
class deep_sea(Environment):
    '''
    Description:
        A deep sea environment, where a diver goes
        down and each time and she needs to make a
        decision to go left or right.
        environment terminates after fixed time step

    Observation:
        [horizontal position, vertical position]

    Actions:
        2 possible actions:
        0 - left
        1 - right

    Starting State:
        start at position 0, time step 0

    Episode termination:
        Env terminates after fixed number of time steps
    '''

    def __init__(self, num_steps):
        '''
        Inputs:
            num_steps: An integer that represents the size of the DeepSea environment
        '''
        self.num_steps = num_steps
        self.epLen = num_steps
        self.flip_mask = 2*np.random.binomial(1,0.5,(num_steps,num_steps))-1
        self.nAction = 2
        self.nState = pow(num_steps,2)
        self.epLen = num_steps
        self.R = {}
        self.states = {}
        for s in range(self.num_steps):
            for s_ in range(self.num_steps):
                self.R[(s,s_), 0] = (0, 0)
                self.R[(s,s_), 1] = (-0.01/self.nState, 0)
                self.states[(s,s_)] = 0
        self.R[(self.num_steps-1,self.num_steps-1),1] = (0.99,0)

    def name(self):
        return  "deep sea"

    def reset(self):
        self.state = (0,0)
        self.timestep = 0
        return copy.deepcopy(self.state)

    def advance(self,action):
        assert action in [0,1], "invalid action"
        self.state_prev = self.state
        step_horizontal = (2*action-1)
        horizontal = max(self.state[0] + step_horizontal, 0)
        vertical = self.state[1] + 1
        done =  bool(vertical == self.num_steps)
        self.state = (horizontal, vertical)
        self.timestep += 1
        return self.R[self.state_prev,action][0], copy.deepcopy(self.state), done

    def argmax(self,b):
        return np.random.choice(np.where(b == b.max())[0])


In [5]:
def proj(x, lo, hi):
    '''Projects the value of x into the [lo,hi] interval'''
    return max(min(x,hi),lo)

In [16]:
class UCRL_VTR(object):
    '''
    Algorithm 1 as described in the paper Model-Based RL with
    Value-Target Regression
    The algorithm assumes that the rewards are in the [0,1] interval.
    '''
    def __init__(self,env,K,random_explore):
        self.env = env
        self.K = K
        # A unit test that randomly explores for a period of time then learns from that experience
        # Here self.random_explore is a way to select a period of random exploration.
        # When the current episode k > total number of episodes K divided by self.random_explore
        # the algorithm switches to the greedy action with respect to its action value Q(s,a).
        if random_explore:
            self.random_explore = 10
        else:
            self.random_explore = self.K
        # Here the dimension (self.d) for the Tabular setting is |S x A x S| as stated in Appendix B
        self.d = self.env.nState * self.env.nAction * self.env.nState
        # In the tabular setting the basis models is just the dxd identity matrix, see Appendix B
        self.P_basis = np.identity(self.d)
        #Our Q-values are initialized as a 2d numpy array, will eventually convert to a dictionary
        self.Q = {(h,s,a): 0.0 for h in range(self.env.epLen) for s in self.env.states.keys() \
                   for a in range(self.env.nAction)}
        #Our State Value function is initialized as a 1d numpy error, will eventually convert to a dictionary
        self.V = {(h,s): 0.0 for s in self.env.states.keys() for h in range(env.epLen + 1)} # self.V[env.epLen] stays zero
        #self.create_value_functions()
        #The index of each (s,a,s') tuple, see Appendix B
        self.sigma = {}
        self.state_idx = {}
        self.createIdx()
        #See Step 2, of algorithm 1
#         self.M = env.epLen**2*self.d*np.identity(self.d)
        # For use in the confidence bound bonus term, see Beta function down below
        self.lam = 1.0
        #Self.L is no longer need, but will keep for now.
        self.L = 1.0
        self.M = np.identity(self.d)*self.lam
        #For use in the Sherman-Morrison Update
        self.Minv = np.identity(self.d)*(1/self.lam)
        #See Step 2
        self.w = np.zeros(self.d)
        #See Step 2
        self.theta = np.dot(self.Minv,self.w)
        #See Step 3
        self.delta = 1/self.K
        #m_2 >= the 2-norm of theta_star, see Bandit Algorithms Theorem 20.5
        self.error()
        # See Theorem 20.5 for m_2
        self.m_2 = np.linalg.norm(self.true_p) + 0.1
#         #Initialize the predicted value of the basis models, see equation 3
#         self.X = np.zeros((env.epLen,self.d))
        #See Assumptions 2,2' and Theorem 1, this equals 1 in the tabular case
        self.C_phi = 1.0
        # See Assumption 2'(Stronger Feature Regularity), and consider the case when v_1 = v_2 = ....
        self.C_psi = np.sqrt(env.nState)
        # See Theorem 1
        self.C_M = 1.0
        # See Theorem 1
        self.C_psi_ = 1.0
        # This value scales our confidence interval, must be > 0
        self.c = 1.0
        self.d1 = env.nState * env.nAction




    def feature_vector(self,s,a,h):
        '''
        Returning sum_{s'} V[h+1][s'] P_dot(s'|s,a),
        with V stored in self.
        Inputs:
            s - the state
            a - the action
            h - the current timestep within the episode
        '''
        sums = np.zeros(self.d)
        for s_ in self.env.states.keys():
            #print(s,s_)
            sums += self.V[h+1,s_] * self.P_basis[self.sigma[(s,a,s_)]]
        return sums

    def proj(self, x, lo, hi):
        '''Projects the value of x into the [lo,hi] interval'''
        return max(min(x,hi),lo)

    def update_Q(self,s,a,k,h):
        '''
        A function that updates both Q and V, Q is updated according to equation 4 and
        V is updated according to equation 2
        Inputs:
            s - the state
            a - the action
            k - the current episode
            h - the current timestep within the episode
        Currently, does not properly compute the Q-values but it does seem to learn theta_star
        '''
        #Here env.R[(s,a)][0] is the true reward from the environment
        # Alex's code: X = self.X[h,:]
        # Suggested code:
        X = self.feature_vector(s,a,h)
        self.Q[h,s,a] = self.proj(self.env.R[(s,a)][0] + np.dot(X,self.theta) + self.Beta(k) \
            * np.sqrt(np.dot(np.dot(np.transpose(X),self.Minv),X)), 0, self.env.epLen )
        self.V[h,s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_Qend(self,k):
        '''
        A function that updates both Q and V at the end of each episode, see step 16 of algorithm 1
        Inputs:
            k - the current episode
        '''
        #step 16
        for h in range(self.env.epLen-1,-1,-1):
            for s in self.env.states.keys():
                for a in range(self.env.nAction):
                    #Here env.R[(s,a)][0] is the true reward from the environment
                    # Alex's code: X = self.X[h,:]
                    # Suggested code:
                    self.update_Q(s,a,k,h)
                self.V[h,s] = max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))

    def update_stat(self,s,a,s_,h):
        '''
        A function that performs steps 9-13 of algorithm 1
        Inputs:
            s - the current state
            a - the action
            s_ - the next state
            k - the current episode
            h - the timestep within episode when s was visited (starting at zero)
        '''
        #Step 10
#         self.X[h,:] = self.feature_vector(s,a,h) # do not need to store this
        X = self.feature_vector(s,a,h)
        #Step 11
        y = self.V[h+1,s_]
#         if s_ != None:
#             y = self.V[h+1][s_]
#         else:
#             y = 0.0
        #Step 12
        self.M = self.M + np.outer(X,X)
        #Sherman-Morrison Update
        self.Minv = self.Minv - np.dot((np.outer(np.dot(self.Minv,X),X)),self.Minv) / \
                    (1 + np.dot(np.dot(X,self.Minv),X))
        #Step 13
        self.w = self.w + y*X

    def update_param(self):
        '''
        Updates our approximation of theta_star at the end of each episode, see
        Step 15 of algorithm1
        '''
        #Step 15
        #print(self.M)
        self.theta = np.matmul(self.Minv,self.w)

    def act(self,s,h,k):
        '''
        Returns the greedy action with respect to Q_{h,k}(s,a) for a \in A
        see step 8 of algorithm 1
        Inputs:
            s - the current state
            h - the current timestep within the episode
        '''
        #step 8
        if k > self.K /self.random_explore:
            #print (max(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)])))
            return self.env.argmax(np.array([self.Q[(h,s,a)] for a in range(self.env.nAction)]))
        else:
            return bernoulli.rvs(0.9) #A random policy for testing

    def createIdx(self):
        '''
        A simple function that creates sigma according to Appendix B.
        Here sigma is a dictionary who inputs is a tuple (s,a,s') and stores
        the interger index to be used in our basis model P.
        '''
        i = 0
        j = 0
        k = 0
        for s in self.env.states.keys():
            self.state_idx[s] = int(j)
            j += 1
            for a in range(self.env.nAction):
                for s_ in self.env.states.keys():
                    self.sigma[(s,a,s_)] = int(i)
                    i += 1

    def Beta(self,k):
        '''
        A function that return Beta_k according to Theorem 20.5 in Bandits book
        Also, if you return np.sqrt(first + second) instead of first + second then
        you get higher cumlative reward. However, in Theorem 19.2, Beta_n is already
        under the square root.
        '''
        #Step 3
        #Bonus as according to step 3
        #return 16*pow(self.m_2,2)*pow(env.epLen,2)*self.d*np.log(1+env.epLen*k) \
        #    *np.log(pow(k+1,2)*env.epLen/self.delta)*np.log(pow(k+1,2)*env.epLen/self.delta)

        #Confidence bound from Chapter 20 of the Bandit Algorithms book, see Theorem 20.5.
        first = np.sqrt(self.lam)*self.m_2
        (sign, logdet) = np.linalg.slogdet(self.M)
        #second = np.sqrt(2*np.log(1/self.delta) + self.d*np.log((self.d*self.lam + k*self.L*self.L)/(self.d*self.lam)))
        det = sign * logdet
        second = np.sqrt(2*np.log(1/self.delta) + np.log(k) + min(det,pow(10,10)) - np.log(pow(self.lam,self.d)))
        return first + second
    
    def Beta_2(self,k):
        '''
        Beta as defined in Mengdi's MatrixRL paper
        '''
        first = self.c*(self.C_M * self.C_psi_ ** 2)
        second = np.log(k*self.env.epLen*self.C_phi)*self.d1
        return np.sqrt(first * second)

    def run(self):
        R = []
        reward = 0.0
        for k in tqdm(range(1,self.K+1)):
            self.env.reset()
            done = 0
            while done != 1:
                s = self.env.state
                h = self.env.timestep
                a = self.act(s,h,k)
                r,s_,done = self.env.advance(a)
                reward += r
                self.update_stat(s,a,s_,h)
            self.update_param()
            self.update_Qend(k)
            R.append(reward)
        return R

    def name(self):
        return 'UCRL_VTR'
    
    def error(self):
        self.true_p = []
        for values in self.env.P.values():
            for value in values:
                self.true_p.append(value)
        print('The 2-norm of (P_true - theta_star) is:',np.linalg.norm(self.true_p-self.theta))

In [17]:
env = make_riverSwim(epLen = 20, nState = 4)
K = 1000
random_explore = False
agent = UCRL_VTR(env,K,random_explore)
R = 0

The 2-norm of (P_true - theta_star) is: 2.513961017995307


In [18]:
#env = deep_sea(num_steps = 4) It works on DeepSea, however, it is VERY SLOW in the Tabular Case.

for k in (range(1,K+1)):
    env.reset()
    done = 0
    while done != 1:
        s = env.state
        h = env.timestep
        a = agent.act(s,h,k)
        r,s_,done = env.advance(a)
        R += r
        #count[s,s_] += 1
        if done != 1:
            agent.update_stat(s,a,s_,h)
    agent.update_param()
    agent.update_Qend(k)
    if k % 100 == 1:
        agent.error()

The 2-norm of (P_true - theta_star) is: 2.513961017995307
The 2-norm of (P_true - theta_star) is: 0.4158972940656462
The 2-norm of (P_true - theta_star) is: 0.3973276245287793
The 2-norm of (P_true - theta_star) is: 0.46717367615703603
The 2-norm of (P_true - theta_star) is: 0.4032683326669099
The 2-norm of (P_true - theta_star) is: 0.3972100315769809
The 2-norm of (P_true - theta_star) is: 0.43926876287208527
The 2-norm of (P_true - theta_star) is: 0.4595539171522595
The 2-norm of (P_true - theta_star) is: 0.4399743662712552
The 2-norm of (P_true - theta_star) is: 0.4140109346364289


In [19]:
true_p = []
for values in env.P.values():
    for value in values:
        true_p.append(value)
print('The 2-norm of (P_true - theta_star) is:',np.linalg.norm(true_p-agent.theta))

'''
Interestingly enough, the old bonus leads to less cumlative reward but a more accurate model.
The new bonus leads to larger cumlative reward at the expense of model accuracy. At least for the
riverswim environment.
'''
print('The cumlative reward is:', R)

The 2-norm of (P_true - theta_star) is: 0.43786200595262564
The cumlative reward is: 8053.755000000039


In [12]:
agent.theta

array([ 9.79354165e-01,  2.30061283e-02, -1.99122661e-03, -4.73048894e-04,
        1.38176269e-01,  9.29986465e-01, -8.17223809e-02,  1.42143573e-02,
        9.55506974e-01,  4.88281914e-02, -3.94880223e-03, -4.94259517e-04,
        1.25640403e-01,  6.50969484e-01,  1.88168941e-01,  4.13301223e-02,
        1.36257845e-02,  9.75714766e-01,  1.32674359e-02, -2.63327380e-03,
        5.60494254e-01, -5.53768031e-01,  7.00985552e-01,  3.00960921e-01,
       -1.19009841e-03,  7.82959917e-03,  9.72763404e-01,  2.05128155e-02,
        8.26069318e-03, -1.42034265e-02,  1.05428487e-01,  9.01252270e-01])